# Sample Code Run for `xagg`
(currently in beta - also this notebook is currently not runnable; the author has to update the environment)

A quick guide to `xagg`, explaining how to use it to aggregate raster data to polygons, with a little bit of info on what goes on under the hood.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import warnings
import xesmf as xe

## Intro

We'll be aggregating a gridded dataset onto a set of shapefiles, using an extra set of weights. Specifically, we'll use:
- gridded: month-of-year average temperature projections for the end-of-century from a climate model (CCSM4)
- shapefiles: US counties
- additional weights: global gridded population density ([GPW](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4), 15 min resolution)

This is a setup that you may for example use if projecting the impact of temperature on some human variable (temperature vs. mortality, for example) for which you have data at the US county level. Since your mortality data is likely at the county level, you need to aggregate the gridded climate model output data to counties - i.e., what is the average temperature over each county? This code will calculate which pixels overlap each county - and by how much - allowing an area-averaged value for monthly temperature at the county level. 

However, you also care about where people live - so you'd like to additionally weight your temperature estimate by a population density dataset. This code easily allows such additional weights. The resultant output is a value of temperature for each month at each county, averaged by both the overlap of individual pixels and the population density in those pixels. 

Let's get started.

In [2]:
# Load some climate data as an xarray dataset
ds = xr.open_dataset('data/climate_data/tas_Amon_CCSM4_rcp85_monthavg_20700101-20991231.nc')

In [3]:
# Load US counties shapefile as a geopandas GeoDataFrame
gdf = gpd.read_file('data/geo_data/UScounties.shp')

In [4]:
# Load global gridded population data from GPW
ds_pop = xr.open_dataset('data/pop_data/gpw_v4_population_density_rev11_15_min.nc')
# Some edits to make sure it's in the right format (an xarray DataArray)
ds_pop = ds_pop['Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 15 arc-minutes']
ds_pop = ds_pop.isel(raster=0)

Now, let's load `xagg` (at least in its current form as just a `python` script full of all the functions)

In [5]:
import xagg as xa

## Calculating area weights between a raster grid and polygons
First, we have to figure out how much each pixel overlaps each polygon. This process requires a few steps:

0. Get everything in the right format. 
    * Gridded data comes in all shapes and sizes. `xagg` is ready to deal with most common grid naming conventions - so no matter if your lat and lon variables are called 'Latitude' and 'Longitude' or 'y' and 'x' or many options in between, as long as they're in xarray Datasets or DataArrays, they'll work. 
    * Behind the scenes, longitude values are also forced to -180:180 (from 0:360, if applicable), just to make sure everything is operating in the same coordinate system. 
1. Build polygons for each pixel 
    * To figure out how much each pixel overlaps each polygon, pixel polygons have to be constructed. If your gridded variable already has "lat_bnds" and "lon_bnds" (giving the vertices of each pixel) explicitly included in the `xr.Dataset`, then those are used. If none are found, "lat_bnds" and "lon_bnds" are constructed by assuming the vertices are halfway between the coordinates in degrees. 
    * If an additional weighting is used, the weighting dataset and your gridded data have to be homogenized at this stage. By default, the weighting dataset is regridded to your gridded data using `xesmf`. Future versions will also allow regridding the gridded data to the weighting dataset here(it's already accounted for in some of the functions, but not all).
    * To avoid creating gigantic geodataframes with pixel polygons, the dataset is by default subset to a bounding box around the shapefiles first. In the aggregating code below, this subsetting is taken into account, and the input `ds` into `xa.aggregate` is matched to the original source grid on which the overlaps were calculated. 
2. Calculate area overlaps between each pixel and each polygon
    * Now, the overlap between each pixel and each polygon is calculated. Using `geopandas`' excellent polygon boolean operations and area calculations, the intersection between the raster grid and the polygon is calculated. For each polygon, the coordinates of each pixel that intersects it is saved, as is the relative area of that overlap (as an example, if you had a county the size and shape of one pixel, but located half in one pixel and half in the other pixel, those two pixels would be saved, and their relative area would be 0.5 each). Areas are calculated using the WGS84 geoid. 
    

In [6]:
# Calculate overlaps
weightmap = xa.pixel_overlaps(ds,gdf,weights=ds_pop)

creating polygons for each pixel...
regridding weights to data grid...
Overwrite existing file: bilinear_720x1440_58x91.nc 
 You can set reuse_weights=True to save computing time.


/Users/kevinschwarzwald/opt/anaconda3/envs/binder_environment/lib/python3.7/site-packages/xesmf/frontend.py:349: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


calculating overlaps between pixels and output polygons...
success!


## Aggregating gridded data to the polygons using the area weights (and other weights) calculated above
Now that we know which pixels overlap which polygons and by how much (and what the value of the population weight for each pixel is), it's time to aggregate data to the polygon level. `xagg` will assume that all variables in the original `ds` that have `lat` and `lon` coordinates should be aggregated. These variables may have extra dimensions (3-D variables (i.e. `lon x lat x time`) are confirmed to work; 4-D etc. should be supported but haven't been tested yet - the biggest issue may be in exporting). 

Since we included an additional weighting grid, this dataset is included in `weightmap` from above and is seamlessly integrated into the weighting scheme.  

In [7]:
# Aggregate
aggregated = xa.aggregate(ds,weightmap)

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating tas...
all variables aggregated to polygons!


## Converting aggregated data
Now that the data is aggregated, we want it in a useable format. 

Supported formats for converting include:
- `xarray` Dataset (using `.to_dataset()`)
    - Grid dimensions from the original dataset are replaced with a single dimensions for polygons - by default called "poly_idx" (change this with the `loc_dim=...` option). Aggregated variables keep their non-grid dimensions unchanged; with their grid dimension replaced as above. 
    - All original fields from the `geodataframe` are kept as `poly_idx x 1` variables. 
- `pandas` Dataframe (using `.to_dataframe()`)
    - All original fields from the `geodataframe` are kept; the aggregated variables are added as separate columns. If the aggregated variables have a 3rd dimension, they are reshaped long - with procedurally generated column names (just `[var]0`, `[var]1`, ... for now). 

(the "raw" form of the geodataframe used to create these can also be directly accessed through `aggregated.agg`)


In [8]:
# Example as a dataset
ds_out = aggregated.to_dataset()
ds_out

<xarray.Dataset>
Dimensions:     (month: 12, pix_idx: 3141)
Coordinates:
  * pix_idx     (pix_idx) int64 0 1 2 3 4 5 6 ... 3135 3136 3137 3138 3139 3140
  * month       (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    NAME        (pix_idx) object 'Lake of the Woods' 'Ferry' ... 'Broomfield'
    STATE_NAME  (pix_idx) object 'Minnesota' 'Washington' ... 'Colorado'
    STATE_FIPS  (pix_idx) object '27' '53' '53' '53' ... '02' '02' '02' '08'
    CNTY_FIPS   (pix_idx) object '077' '019' '065' '047' ... '240' '068' '014'
    FIPS        (pix_idx) object '27077' '53019' '53065' ... '02068' '08014'
    tas         (pix_idx, month) float64 263.6 268.5 273.8 ... 283.5 276.4 270.4

In [9]:
# Example as a dataframe
aggregated.to_dataframe()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,tas0,tas1,tas2,tas3,tas4,tas5,tas6,tas7,tas8,tas9,tas10,tas11
0,Lake of the Woods,Minnesota,27,077,27077,263.602520,268.545873,273.792061,283.052187,290.588345,297.902315,302.227761,300.547098,293.499588,283.709873,274.934804,265.811234
1,Ferry,Washington,53,019,53019,271.647085,275.494237,276.804244,279.711224,286.513667,293.616388,298.871477,296.919509,289.612802,281.479429,276.584659,272.100305
2,Stevens,Washington,53,065,53065,273.542482,277.244200,278.724128,281.575573,287.979796,295.091591,301.209235,299.693608,292.715863,283.625825,278.371373,274.006783
3,Okanogan,Washington,53,047,53047,271.763197,275.534344,276.663923,279.341413,285.840061,292.728334,297.850271,296.019671,289.161385,281.395078,276.602879,272.204725
4,Pend Oreille,Washington,53,051,53051,272.843268,276.590636,278.059846,281.031699,287.637497,294.738294,300.555274,298.861150,291.707076,282.870127,277.708402,273.272679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,Skagway-Hoonah-Angoon,Alaska,02,232,02232,274.960020,276.663175,277.738008,279.890377,283.905203,288.146291,289.779068,289.810510,286.806806,282.143341,278.313569,275.517545
3137,Yukon-Koyukuk,Alaska,02,290,02290,264.523545,264.077912,267.580280,273.396936,281.620129,289.282652,288.869535,286.155696,280.782178,273.674911,266.709987,265.538477
3138,Southeast Fairbanks,Alaska,02,240,02240,263.571885,263.654951,266.522257,271.981801,279.840355,287.741933,288.055385,285.488253,279.716905,272.268435,265.532520,264.062459
3139,Denali,Alaska,02,068,02068,264.921667,264.594739,267.760523,272.976810,280.327381,288.373121,288.447619,285.864451,280.229201,272.847219,266.304625,265.445692


## Exporting aggregated data
`xagg` can also directly export the aggregated data to files 

Supported export options include: 

- NetCDF (using `.to_netcdf(fn)`)
    - `lon` and `lat` are replaced with a single `poly_idx` (you can change this name for the dimension in the `prep_for_nc` or `output_data` functions). All original fields from the input shapefile are kept as variables.
    - This is likley the format most useful for further climatological work, in python, Matlab, etc.
- csv (using `.to_csv(fn)`)
    - Rows are polygons (counties in this case) and columns are variables. All original fields from the input shapefile are kept as columns (e.g. FIPS codes in the county data); the data variables are then added "wide", with procedurally generated names for elements of extra dimensions. For example, in this case, the temperature data (called `tas` in the input `ds`) is `lon x lat x month`, with 12 months in the `month` dimension. The values for each of these months are then saved as columns `tas0`, `tas1`, ... `tas11`. Future versions may include more sophisticated column naming options (dynamic date naming if it recognizes a date dimension, for example). 
    - This is likely the format most useful for use in STATA or R
- shapefile (using `.to_shp(fn)`)
    - Data variables are saved as fields in the shapefile, with one field per member of the 3rd dimension (if applicable) similar to the .csv format above
    - This is likely the format most useful for use in QGIS, etc.

In [10]:
# Now let's export it as a .csv (from the dataframe generated as above) 
aggregated.to_csv('tas_Amon_CCSM4_UScounty_aggregated')

tas_Amon_CCSM4_UScounty_aggregated.csv saved!


# Sample figures

Let's see what this aggregated data looks like! Let's make a figure comparing the original inputted grid January average temperature with the aggregated January average temperature.

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import cartopy 
import cartopy.crs as ccrs
import shapely
%matplotlib inline

In [ ]:
gdf['tas'] = ds_out.tas.isel(month=0).values

In [ ]:
hist_bins = np.arange(273.15-30,273.15+30,2.5)
hist_idxs = np.digitize(gdf.tas,hist_bins)

norm = plt.Normalize(273.15-30, 273.15+30)

proj = ccrs.AlbersEqualArea(central_longitude=-100)

cmap = mpl.cm.RdBu_r
color = cmap(norm(hist_bins))

fig = plt.figure(figsize=(15,5))

ax0 = plt.subplot(1,2,1,projection=proj)
ds.isel(month=0).sortby('lon').tas.plot(transform=ccrs.PlateCarree(),
                                        vmin=273.15-30,vmax=273.15+30,
                                        levels=int(60/2.5)+1,
                                        cmap=mpl.cm.RdBu_r,
                                        add_colorbar=False)
ax0.set_extent([-170,-65,10,80])
ax0.coastlines()
ax0.set_title('Original grid',fontsize=20)


ax1 = plt.subplot(1,2,2,projection=proj)

for hist_digit in np.unique(hist_idxs):
    ax1.add_geometries(gdf.to_crs(proj.proj4_init).iloc[hist_idxs==hist_digit]['geometry'],
                       crs = proj,
                       facecolor=(color[hist_digit]))

ax1.set_extent([-170,-65,10,80])
ax1.coastlines()
ax1.set_title('Aggregated data',fontsize=20)


fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.9, 0, 0.05, 1])
#norm = mpl.colors.Normalize(vmin=-30, vmax=30)
norm = mpl.colors.BoundaryNorm(np.linspace(-30,30,int(60/2.5)+1), cmap.N)
cbar_ax.axis('off')
cb1 = plt.colorbar(mpl.cm.ScalarMappable(norm=norm,cmap=cmap),ax=cbar_ax,
                   aspect=30)
cb1.set_label('mean January temperature\n($^\circ$C, 2070-2099, CCSM4)',fontsize=15)

# Step-by-step Run
Here is a step-by-step run, bypassing the wrapper functions to explain what the code is doing behind the scenes. `ds2` in each is just to allow investigating intermediate results.

In [11]:
# Put dataset into the right format (fix lat/lon grid names and remap to -180:180 lon)
ds2 = xa.fix_ds(ds)

In [12]:
# Check to see if latitude/longitude bounds are included in the dataset, and if not, 
# create them from scratch by assuming each pixel extends halfway between coordinates
ds2 = xa.get_bnds(ds2)

In [13]:
# Process weights - if weights are used, they are either regridded to the ds grid 
# (target="ds") or the ds grid is regridded to the weights set (target="weights" - 
# not yet fully implemented). (nothing happens if weights stays None)
ds2 = xa.core.process_weights(ds2,weights=ds_pop,target="ds")

regridding weights to data grid...
Overwrite existing file: bilinear_720x1440_192x288.nc 
 You can set reuse_weights=True to save computing time.


/Users/kevinschwarzwald/opt/anaconda3/envs/binder_environment/lib/python3.7/site-packages/xesmf/frontend.py:349: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


In [14]:
# Create raster polygons (this function actually runs fix_ds, get_bnds, and 
# process_weights). subset_bbox = [gdf] means that only the pixels within the 
# bounding box around the shapefiles (plus a safety distance of a bit more
# than the width of the widest pixel) are processed.
pix_agg = xa.core.create_raster_polygons(ds,weights=ds_pop,weights_target="ds",
                                 subset_bbox=gdf)

regridding weights to data grid...
Overwrite existing file: bilinear_720x1440_58x91.nc 
 You can set reuse_weights=True to save computing time.


/Users/kevinschwarzwald/opt/anaconda3/envs/binder_environment/lib/python3.7/site-packages/xesmf/frontend.py:349: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs


In [15]:
# Now calculate the overlap between the raster polygons and the shapefile
# polygons
wm_tmp = xa.core.get_pixel_overlaps(gdf,pix_agg)

In [16]:
# Find subset of ds that matches the subsetted ds (subset_bbox = True above)
# used to create the raster polygons (this function is run in [aggregate] 
# below)
ds2 = xa.subset_find(ds,wm_tmp.source_grid)

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful


In [17]:
# Now, aggregate the original data variables onto the polygons (this contains
# subset_find above)
agg2 = xa.aggregate(ds,wm_tmp)

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())
grid adjustment successful
aggregating tas...
all variables aggregated to polygons!
